---
title: "Theory of RL I: Intro to Reinforcement Learning"
date: "2021-04-15"
image: "image.png"
format:
  html:
    code-fold: show
    code-line-numbers: false
jupyter: python3
---

Reinforcement learning (RL) is the task of **learning how to behave in an uncertain environment by interacting with it in order to achieve a goal**. In that sense, RL closely resembles the way humans learn: while supervised learning means learning from labeled examples and unsupervised learning is concerned with finding patterns in unlabeled data, RL is tasked with **solving sequential decision-making problems under uncertainty**.

In RL, the learning algorithm is called **agent** and interacts over a **sequence of discrete time steps** with an **environment** that may be incompletely controllable as well as incompletely known. At each time step, the agent receives a representation of the environment's **state** which he uses to select an **action**. In the following time step, the environment has reached a new state and the agent gets a numerical **reward** signal to help him evaluate his choice of action. The agent's goal is to maximize the cumulative reward received by developing an optimal **policy**, i.e. an optimal strategy for selecting actions depending on the state of the environment. This continued interaction can be formalized by a mathematical framework called **Markov decision process (MDP)** that serves as the theoretical foundation of RL algorithms. Its components are introduced below.

## Terminology

### Agent

An agent is a **decision maker** in an environment. It can sense at least parts of that environment and it can choose actions to influence that environment. Its goal is to maximize cumulative rewards by finding the optimal strategy to navigate the environment. Thus, an agent can be seen as following a process with three steps:

1. Interaction with the environment (by observing its state and choosing an action)
2. Evaluation of the current behavior
3. Improvement of the behavior to fare better in the future

In practice, the decision-making algorithm serves as the agent.

### Environment

The environment bundles together everything outside of the agent and **embodies the problem or task that the agent faces**. Therefore, it is crucial to think about the boundary between agents and the environment. Agents are decision makers *only*. **Everything an agent has no absolute control over** (i.e. cannot change arbitrarily) belongs to its environment. For example, even the sensors, motors, arms, etc. of a robot are typically considered to be part of the environment. The agent in this case is only the algorithm that controls the robot. This does *not* mean that everything in the environment is *unknown* to the agent, however. The agent can observe at least parts of the environment and may even know exactly which rules govern the environment's dynamics.

### State and Observation

An environment can be represented by a set of variables called the **state**. The state space $\mathcal{S}$ (itself a set) defines which values those variables can possibly take. A state $s \in \mathcal{S}$ is simply an instantiation of the state space, i.e. a **unique configuration of the environment** at a particular point in time. While $\mathcal{S}$ can be finite or infinite, the set of variables that compose a single state $s$ has to be finite and of equal size for every state. The state of the environment at time step $t$ is denoted $S_t$.

In the MDP framework, we assume that the states 1) are fully observable, i.e. fully capture the internals of the environment, and 2) contain all variables required to make them independent of all other states, i.e. they are a self-contained representation of the problem. If an agent – in violation of that assumption – can only partially observe the environment, the information available to the agent is called an **observation** $o$. The set of all possible values an observation can take is then called the observation space $\mathcal{O}$. This is a more general (and more realistic) framing of the problem that can be seen as an extension of the MDP framework. It is referred to as POMDP (Partially Observable Markov Decision Process). It should be noted, though, that both terms (state and observation) are often used interchangeably despite their slightly different meanings.

### Action

At every time step, an agent can either deterministically or stochastically select an action $a$ to **influence the environment**. The set of all possible actions is called the action space $\mathcal{A}$ and can be finite or infinite. It is possible that not every action is available in every state and that an action doesn't affect the environment at all (i.e., a no-op action, meaning that the environment stays in the same state). The action selected at time step $t$ is denoted $A_t$.

### Transition function

In response to the action of an agent **the environment may change its state** ("may" because it can also stay in the same state). The transition to a new state $s'$ depends on the old state $s$ and an action $a$ and is either deterministically or stochastically defined by the state-transition function $p(s' \vert s, a)$ which outputs the probability of the transition from state $s$ to $s'$ when taking action $a$. 

$$p(s' \vert s, a) = P(S_t = s' \vert S_{t-1} = s, A_{t-1}=a)$$

$$\sum_{s' \in\ \mathcal{S}} p(s' \vert s, a) = 1, \forall s \in \mathcal{S}, \forall a \in \mathcal{A}(s)$$

Many RL algorithms assume that the probability distribution of transitions does not change over time. However, this **stationarity assumption** has to be relaxed in many situations.

### Reward function

The environment responds to an interaction with an agent by providing a numerical reward which **signals the goodness of the transition**. Rewards can be negative, in which case they can be interpreted as cost or penalty. Since it is the objective of an agent to maximize cumulative rewards, the reward function **defines the goal of an RL task**. Hence, the **design of the reward scheme is critical** when constructing an RL problem. While a sparse reward signal offers less supervision and thus a higher chance of novel, unexpected behavior, a more dense signal usually allows the agent to learn faster, but also injects bias. Usually, the injection of prior knowledge is not something we want: the reward function should only tell the agent *what* he should achieve and not *how* he should achieve it. However, adding a time step cost is a common practice.

The reward function is typically defined as a function that takes the full transition tuple $s,a,s'$ and outputs the expected reward ("expected" since reward signals may be stochastic):
$$r(s,a,s') = \mathbb{E} \left[ R_t\ \vert\ S_{t-1} = s, A_{t-1} = a, S_t = s' \right]$$

Note that the reward signal received at time step $t$, $R_t$, not only depends on the state and the action in the last time step $t{-}1$, but also on the next state $S_t$. This makes sense: for example, success in a game of tennis not only depends on the state of the game and your shot selection (the action) in the last time step, but also (and rather importantly) on the outcome of your shot (the next state). 

Also note that computing the marginalization over next states gives $r(s,a)$...
$$r(s,a) = \mathbb{E} \left[R_t \vert S_{t-1} = s, A_{t-1} = a \right]$$

...and further marginalization over actions gives $r(s)$:
$$r(s) = \mathbb{E} \left[R_t \vert S_{t-1} = s \right]$$

### Dynamics of the MDP

Having introduced states, actions, and rewards, we can now look at the function $p(s',r \vert s, a)$ which completely defines the **dynamics** of an MDP:

$$p(s',r \vert s, a) = P(S_t = s', R_t = r\ \vert\ S_{t-1} = s, A_{t-1}=a)$$

$$\sum_{s' \in \mathcal{S}} \sum_{r \in \mathcal{R}} p(s',r \vert s,a) = 1, \forall s \in \mathcal{S}, \forall a \in \mathcal{A}(s)$$

The formula shows that in an MDP the probability of each possible value for $S_t$ and $R_t$ depends *only* on the state and action in the last time step. That is, the probability of transitioning from state $s$ to $s'$ given action $a$ has to be the same regardless of all states and actions encountered before. Put simply, **the future and the past are conditionally independent *given the present*** because the present state encapsulates all necessary information. This is called the **Markov property**.

Note that we can quite easily compute the state-transition function $p(s' \vert s, a)$ as well as the reward function $r(s,a,s')$ from the dynamics function $p(s', r \vert s, a)$:

$$p(s' \vert s, a) = \sum_{r \in \mathcal{R}} p(s', r \vert s, a)$$

$$r(s,a,s') = \sum_{r \in \mathcal{R}} r \dfrac{p(s',r \vert s, a)}{p(s' \vert s, a)}$$

If an agent has a perfect **model** of the environment's dynamics, we can, in theory (i.e. disregarding practical issues of computational power and memory), find the optimal solution of an RL task. If there is only incomplete information available we have to approximate the optimal solution.

### Episodic vs. Continuing Tasks

In RL, we distinguish between episodic and continuing tasks. **Episodic tasks have a natural ending**, that is, they end in a **terminal state** after which the environment can be reset and the agent has to start from the beginning. A terminal state is a special state that transitions only to itself while providing no reward. To handle episodic tasks, we need to introduce some new notation: while the set of all nonterminal states is denoted $\mathcal{S}$, the set of all states including terminal states is typically denoted $\mathcal{S}^+$. The time step when an **episode** terminates is denoted $T$ and usually varies from episode to episode.

When the interaction of agent and environment does not break naturally into subsequences but goes on continually without an end, the task is **continuing**. Examples might be the control of an on-going process or the trading of stocks. In continuing tasks $T=\infty$, although in many cases a time step limit is set which results in an episodic task.

### Return

The idea of **cumulative reward** is formalized by the return which is defined as the **function of future rewards that an agent sets out to maximize *in expectation***. While rewards only relay an immediate sense of goodness, returns indicate the performance in the long run. In the simplest case the return is just the sum of the reward sequence in an episode:

$$G_t = R_{t+1} + R_{t+2} +  \dots + R_T$$

However, there are several reasons why we would like to **discount the value of rewards over time**, especially in continuing tasks with infinite reward sequences:

* Reducing the variance of the reward sequence (since future rewards have higher uncertainty, just think about the stock market)
* Focusing on more immediate rewards (since rewards we get sooner may be more attractive than rewards we get later)
* Mathematical and computational convenience (since we don't need to track the far future)


Thus, the standard formula for the return looks like this:
$$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots\ = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$$

The parameter $\gamma$, $0 \leq \gamma \leq 1$, is called the **discount rate** and defines how we value future rewards. A reward of $+1$ received in $k$ time steps is worth only $\gamma^{k-1}$ in the present. This means that an agent maximizes only immediate rewards if $\gamma=0$ (the agent is called "myopic" in this case), and becomes more farsighted as $\gamma$ approaches $1$.

Finally, note that the return can be defined **recursively**:
$$G_t = R_{t+1} + \gamma G_{t+1}$$

### Policy function

A policy, denoted $\pi$, is the **behavior function of an agent**: it **maps states of the environment to actions to be taken in those states** and is thus the key feature of an agent.

In the deterministic case, a policy just maps a state to an action:
$$\pi(s)=a$$

More common is the stochastic case, though. Here, the policy defines a **probability distribution over all available actions for every state**:
$$\pi(a\ \vert\ s) = P_{\pi}(A=a\ \vert\ S=s)$$

A policy $\pi$ is defined to be better than or equal to another policy $\pi'$ if its expected return is greater than or equal to that of $\pi'$ for all states. There is always at least one **optimal policy** $\pi_*$ which is better than or equal to all other policies. Consequently, we want to design an agent that generates (or at least approximates) an optimal policy.

### State-value function

Values form the core of RL. While a reward solely relays an immediate notion of goodness, a value signals what is good in the long run. The **value of a state** is defined as the **return an agent can *expect* to get if he starts in that state and follows policy $\pi$**. Formally, the state-value function $v_{\pi}$ is defined as:
$$v_{\pi} = \mathbb{E}_{\pi} \left[ G_t \vert S_t = S \right] = \mathbb{E}_{\pi} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}\ \middle|\ S_t = s \right]\ \ \forall s \in \mathcal{S}$$

Evidently, agents shouldn't seek to reach states that merely offer high rewards; instead they should go for states with high values (because those states are regularly followed by states with high rewards and thus yield high expected returns). In other words, state-values indicate the long-term desirability of states.

Interestingly (and importantly), $v_{\pi}$ can be expressed **recursively**:

$$\begin{align}
v_{\pi} & = \mathbb{E}_{\pi} \left[ G_t \vert S_t = S \right]\\
        & = \mathbb{E}_{\pi} \left[ R_{t+1} + \gamma G_{t+1} \vert S_t = S \right]\\
        & = \sum_{a} \pi(a \vert s) \sum_{s'} \sum_{r} p(s',r \vert s, a) \left[ r + \gamma \mathbb{E}_{\pi} [ G_{t+1} \vert S_{t+1} = s' ] \right]\\
        & = \sum_{a} \color{orange}{\pi (a \vert s)} \color{black}{\sum_{s', r}} \color{purple}{p(s', r \vert s, a)} \color{blue}{\left[r + \gamma v_{\pi}(s') \right]} \color{black}{,\ \forall\ s \in \mathcal{S}}
\end{align}$$

Equation (4) is called the **Bellman equation**. It is based on the recursive relationship between the value of a state and the value of its possible successor states. Let's go over the math in detail. The Bellman equation is a sum over all possibilities of the action $a$, the next state $s'$, and the reward $r$. For each triple $(a,s',r)$, we calculate the value by summing the reward $\color{blue}{r}$ and the discounted value of the next state $\color{blue}{\gamma v_{\pi}(s')}$. We weight each of those values by its probability of occurring $\color{orange}{\pi (a \vert s)}\color{purple}{p(s', r \vert s, a)}$ (where $\color{orange}{\pi (a \vert s)}$ is the probability of taking action $a$ when following policy $\pi$, and $\color{purple}{p(s', r \vert s, a)}$ is the probability of reaching the next state $s'$ and getting reward $r$ given the current state $s$ and action $a$) and sum over all weighted values to get the expected value of state $s$. Put simply, the value of state $s$ is equal to the expected reward plus the discounted value of the expected next state.

The **optimal state-value function** $v_*$ is the state-value function with the highest value across all policies (or in other words, the largest expected return achievable by any policy):

$$v_*(s) = \max_{\pi} v_{\pi}(s), \forall s \in \mathcal{S}$$

Since $v_*$ is shared by all optimal policies $\pi_*$, it can be expressed without referencing any specific policy:

$$v_*(s) = \color{brown}{\max_a} \color{black}{\sum_{s',r}} \color{purple}{p(s', r \vert s, a)} \color{blue}{\left[ r + \gamma v_*(s') \right]}$$

This is called the **Bellman optimality equation** and expresses the simple fact that, when following an optimal policy, the value of a state must equal the expected return for the best action from that state. Let's go over the math in detail again. To calculate the optimal value of the state $s$ we look at every action $a$ that is available in that state. For each of these actions we look at all possibilities for $s'$ and $r$, compute their value by adding $\color{blue}{r}$ and the discounted *optimal* value of the next state $\color{blue}{\gamma v_*(s')}$, and weight by their probability of occurring $\color{purple}{p(s', r \vert s, a)}$. Finally, to get the optimal value of $s$, we just take the $\color{brown}{\max}$ over all actions.

Given $v_*$, it is easy to determine an optimal policy. In each state $s$, there is at least one action that obtains the maximum in the Bellman optimality equation. Any policy that only chooses between these actions (that is, assigns zero probability to all other actions) is optimal. In other words, **an optimal policy is greedy with respect to the optimal value function** $v_*$.

In principle, assuming we have perfect knowledge of the environment's dynamics, we could solve the system of equations given by the Bellman optimality equation (which gives one equation for each state) to get an exact solution of the RL problem. In practice, however, we usually lack the computational power and/or memory capacity to do so (think of chess, for example), which is why in nontrivial problems we need to settle for approximations. Indeed, most RL algorithms involve estimating a value function: either the state-value function or the action-value function which is introduced in the next section.

### Action-value function

Analogous to the state-value function $v_{\pi}$, the action-value function $q_{\pi}(s,a)$ defines the value of taking action $a$ in state $s$ under policy $\pi$ as the **expected return when starting from $s$, taking action $a$, and *thereafter* following policy** $\pi$:

$$q_{\pi}(s,a) = \mathbb{E}_{\pi} \left[ G_t \middle| S_t = s, A_t = a \right] = \mathbb{E}_{\pi} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}\ \middle|\ S_t = s, A_t = a \right]$$

While state-value functions estimate how good it is for an agent to be in a given state, action-value functions estimate how good it is to take a given action in a given state. Evidently, an agent should take the actions with the highest action-values in order to maximize his expected return.

Again, we can express $q_{\pi}(s,a)$ **recursively**:

$$\begin{align}
q_{\pi}(s,a) & = \mathbb{E}_{\pi} \left[ G_t \vert S_t = S, A_t = a \right]\\
             & = \mathbb{E}_{\pi} \left[ R_{t+1} + \gamma G_{t+1} \vert S_t = S, A_t = a \right]\\
             & = \sum_{s', r} \color{purple}{p(s', r \vert s, a)} \color{blue}{\left[r + \gamma v_{\pi}(s') \right]} \color{black}{,\forall s \in \mathcal{S},\ \forall a \in \mathcal{A}(s)}
\end{align}$$

This gives us the **Bellman equation for action values**: for all state-action pairs we calculate the sum of the reward $\color{blue}{r}$ and the discounted value of the next state $\color{blue}{\gamma v_{\pi}(s')}$, and weight it by the probability $\color{purple}{p(s',r \vert s, a)}$ of reward $r$ and next state $s'$ given state $s$ and action $a$  before summing up. Note that we don't weight over actions since we only look at one specific action.

The **optimal action-value function** $q_*$ is the action-value function with the highest action-values across all policies:

$$q_*(s,a) = \max_{\pi} q_{\pi}(s,a), \forall s \in \mathcal{S}, \forall a \in \mathcal{A}(s)$$

Since $q_*$, like $v_*$, is shared by all optimal policies $\pi_*$, it can be expressed without reference to any specific policy:

$$\begin{align}
q_*(s,a) & = \mathbb{E} \left[R_{t+1} + \gamma \max_{a'} q_*(S_{t+1}, a')\ \middle|\ S_t = s, A_t = a \right]\\
         & = \sum_{s',r} \color{purple}{p(s',r \vert s, a)} \color{darkgreen}{\left[ r + \gamma \max_{a'} q_*(s',a') \right]}
\end{align}$$

This is the **Bellman optimality equation for** $q_*$. To compute the optimal value of taking action $a$ in state $s$, we look at all possible pairs of the next state $s'$ and reward $r$, compute their values by summing the reward $\color{darkgreen}{r}$ and the discounted optimal value of the best action in the next state $s'$ $\color{darkgreen}{\gamma \max_{a'} q_*(s',a')}$, weight by their respective probability $\color{purple}{p(s',r \vert s, a)}$, and sum them up.

Given $q_*$, it is even easier to determine an optimal policy. We don't need to look ahead to possible next states, simply choosing any action that maximizes $q_*(s,a)$ suffices. 

It should be clear now why values (state-values and action-values) are essential for making and evaluating decisions, and why trying to efficiently estimate them plays an important role in most RL algorithms.

### The RL Cycle

Let's summarize the interaction between the agent and the environment:

1. The agent observes the environment. If he gets a perfect representation of the environment, he knows the state $s$ of the environment. If he receives only a partial representation, he has an observation $o$ which depends on the true state $s$.
2. Based on the state or observation, the agent takes a specific action $a$ as determined by his policy $\pi$. Ideally he chooses his action so that he maximizes the expected return $G$.
3. In response to the action of the agent the environment transitions from the old state $s$ to a new state $s'$ (which can be equal to the old state) and provides a reward $r$ to the agent. The set of the state, the action, the reward, and the new state is called an **experience**.
4. The agent tries to learn from this experience to make better decisions in the future.

Typically, these interactions go on for several (possibly infinitely many) time steps, resulting in a so-called **trajectory** $S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, R_3, \dots$.

### Conclusion: RL and MDPs

The MDP framework essentially reduces the RL problem of goal-directed learning from interaction to three signals which are passed from the agent to the environment or vice versa: the actions which represent the agent's decisions, the states which inform the agent's decision making, and the rewards which define the agent's goal. While this is surely idealized, it allows us to model a broad range of sequential decision making problems and even proves to be useful in cases where the formal assumptions of the framework are not exactly met.

## Key Challenges in RL

Now that we have introduced the MDP framework, we can look ahead at the major challenges we face when tackling RL problems:

* **Evaluative feedback**: Instead of instructive feedback which provides the correct action to take (think of supervised learning), an RL agent gets evaluative feedback by the environment. That is, the feedback signal only indicates the goodness but not the correctness of an action. To reach his goal, an agent has to learn good behavior by *exploring* the realm of possible actions, while also *exploiting* his current knowledge by taking the best action he is aware of. This gives rise to the **exploitation vs. exploration trade-off** which we'll discuss in more depth in the next post.
* **Delayed feedback**: Rewards for actions may manifest only several time steps after the corresponding action was chosen by the agent. Thus, an agent has to correctly assign credit to actions even though they have delayed consequences. This is called the **temporal credit assignment problem**.
* **Sampled feedback**: In stochastic tasks the reward provided by the environment is merely a sample. When the agent doesn't have perfect knowledge of the MDP's dynamics, he needs to find a way to get a reliable estimate of the reward he is going to receive.